[Retour au sommaire](../index.ipynb)

# Traitement de données en tables : TP recherche

Nous allons dans travailler sur les données des collèges du finistère.
<div class = "alert alert-info">

Ouvrir votre IDE Python (**I**ntegrated **D**evelopement **E**nvironment) et **sauvegarder en college1.py**.
</div>

- Créer un python virtuel 'TPcsv'. ([voir procédure](../../../Pratique_commune/venv.ipynb))
- Activer le venv TPcsv

## Lecture de l'ensemble des données

In [ ]:
# importation des librairies nécessaires
import requests
import os
import csv
# recuperation des données depuis l'url
url = "https://www.data.gouv.fr/fr/datasets/r/a1075259-09ca-4c22-9e35-b2cd6bbf5c36"
data = requests.get(url).content
# création d'un fichier et écriture des données
local_file = "college_finistere.csv"
with open(local_file, 'wb') as csvfile:
    csvfile.write(data)
# ouverture du fichier précédemment créé
with open(local_file) as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        print(row)

**Remarques**

- Le fichier que l'on ouvre est en mode **bytes** car nous recevons les données sous forme d'un flux d'octets et non de caractères.
- On utilise l'instruction *with* qui garantit que si une erreur se produit dans le with, le fichier sera correctement fermé. [voir l'instruction with](https://docs.python.org/fr/3/reference/compound_stmts.html#the-with-statement)

**A Faire**

<div class="alert alert-info">

- Si le fichier a déjà été téléchargé, grace au module [os](https://docs.python.org/fr/3/library/os.html), trouver une solution pour ne pas le retélécharger.
  
</div>

- Quel est le type des lignes ?
- Pourquoi chaque ligne ne possède qu'un élément ? (print(len(row))
- Quel est le délimiteur de colonnes utilisé dans le fichier ?
- Quel est l'origine du problème sur le nombre d'éléments de chaque ligne ?
- Modifier votre code pour obtenir le nombre correct d'éléments.
- Modifier votre code afin d'obtenir les lignes sous forme de dictionnaires. Voir La [librairie csv](https://docs.python.org/fr/3/library/csv.html).

<div class="alert alert-info">

- Sauvegarder le fichier college1.py. Puis le sauvegarder en  **college2.py**
  
</div>

## Filtre par lignes

- Afficher uniquement les lignes dont le college se trouve à CHATEAULIN ou à PLEYBEN. (Rendre la recherche insensible à la **[casse](https://fr.wikipedia.org/wiki/Casse_(typographie))** de caractères.)
- Afficher uniquement les collèges publics
- Afficher les collèges privés de la métropole brestoise (Brest, Bohars, Gouesnou, Guilers, Guipavas, Le Relecq-Kerhuon, Plougastel-Daoulas, Plouzané)

<div class="alert alert-info">

- Sauvegarder le fichier college2.py. Puis le sauvegarder en  **college3.py**
</div>

## Filtre par colonnes
- Afficher toutes les colonnes SAUF le fax et la geolocalisation. (On partira de l'intégralité des données pour supprimer les deux colonnes)

<div class="alert alert-info">

- Sauvegarder le fichier college3.py. Puis le sauvegarder en  **college4.py**
</div>

## Recherche de doublons

Les fichiers csv peuvent parfois contenir des doublons, il est alors utile de les détecter et de les supprimer.

Nous considérons qu'il ne peut y avoir deux collèges de même nom, dans une même ville.

Pour les besoins de l'exercice, modifiez le fichier csv d'origine afin d'y ajouter plusieurs doublons.

Vous ajouterez également un collège ayant le même nom qu'un autre mais dans une ville différente.

Le but est d'écrire la liste sans les doublons dans un nouveau fichier, par exemple  *"college_finistere_sans_doublon.csv"*.

Le principe est le suivant:

- on ouvre en lecture notre fichier college_finistere.csv
- on ouvre en écriture un fichier college_finistere_sans_doublon.csv
- on crée une liste vide 'liste_ville_college'
- on boucle sur les lignes de notre csv
  - si le tuple (ville, college) n'est pas présent dans la liste 'liste_ville_college'
     - on ajoute le tuple dans la liste
     - on ecrit la ligne dans notre csv college_finistere_sans_doublon.csv

Voici un exemple de code que vous devez adapter.

In [ ]:
import csv
with open('first.csv', 'rb') as entree, open('first_edit.csv', 'wb') as sortie:
    writer = csv.writer(sortie)
    for row in csv.reader(entree):
        if row[2] != "0":
            writer.writerow(row)

Evidemment, vous devez obtenir un fichier csv *college_finistere_sans_doublon.csv* qui ne possède plus les doublons que vous aviez ajoutés.

## Tests de cohérence
<div class="alert alert-info">Sauvegarder le fichier college4.py. Puis le sauvegarder en  <strong>college5.py</strong></div>

Dans un fichier csv, chaque colonne doit être d'un type unique. (Chaine de caractère, entier, date....)

Nous allons vérifier si l'ensemble des codes postaux sont bien des entiers compris entre 29000 et 29999.

Modifier quelques codes postaux afin qu'ils ne respectent pas ces critères.

Les lignes comportant des erreurs devront être copiées dans une liste et affichées à l'utilisateur du programme (vous...).



In [ ]:
import os
import csv
import requests
import folium
from folium import plugins
from folium import FeatureGroup, LayerControl, Map, Marker

def get_icon(row):
    """
    Retourne l'icone adequat
    """
    if row['tarifentree'] == 'Gratuit':
        icon=folium.Icon(color='red', icon='home', prefix='fa')
    elif row['tarifentree'] == 'Payant':
        icon=folium.Icon(color='green', icon='home', prefix='fa')
    elif row['tarifentree'] == 'Tarifs non communiqués':
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    else:
        icon=folium.Icon(color='black', icon='home', prefix='fa')
    return icon



# le fichier est volumineux, à télécharger une seule fois
local_file = "geolocalisation.csv"
if not local_file in os.listdir():
    url = "https://www.data.gouv.fr/fr/datasets/r/d7f3ddf4-2225-4ac2-9a1d-26971ce92969"
    #url = "https://www.data.gouv.fr/fr/datasets/r/7c0f7980-1804-4382-a2a8-1b4af2e10d32"
    data = requests.get(url).content
    with open(local_file, 'wb') as csvfile:
        csvfile.write(data)
        
# Calcul de la position moyenne pour pouvoir centrer la carte
with open(local_file) as csvfile:
    somme_latitude = 0
    somme_longitude = 0
    compteur = 0
    reader = csv.DictReader(csvfile, delimiter=';')
    m = Map(location=[0,0], width="%100", height="%100", zoom_start=11)
    minimap = plugins.MiniMap()
    m.add_child(minimap)
    tooltip = "Click me!"
    groups = {}
    tarif_entree_types = []
    for row in reader:
        code_postal = row['detailidentadressecp']
        code_postal = int(''.join(code_postal.split(' ')))
        if code_postal in [29570, 29160, 29560, 29550, 29150, 29590, 29190]:
            latitude = row['gmaplatitude']
            longitude = row['gmaplongitude']
            if latitude and longitude:
                #print(row)
                tarif_entree = row['tarifentree']
                if tarif_entree not in tarif_entree_types:
                    tarif_entree_types.append(tarif_entree)
                    groups[tarif_entree] = FeatureGroup(name=tarif_entree)
                compteur += 1
                #print(float(row['latitude']) + ';' + float(row['longitude']))
                somme_latitude += float(latitude)
                somme_longitude += float(longitude)
                tooltip = row['syndicobjectname']
                icon = get_icon(row)
                Marker([latitude, longitude], icon=icon, popup="<i>Hello</i>", tooltip=tooltip).add_to(groups[tarif_entree])
    for group in groups.values():
        print(group)
        group.add_to(m)
    LayerControl().add_to(m)
    print(tarif_entree_types)
    moy_latitude = somme_latitude/compteur
    moy_longitude = somme_longitude/compteur
    m.location = (moy_latitude, moy_longitude)
    m.zoom_start=13
m

<table style="width:100%">
    <tr>
        <td style="text-align:left;"><a href="1_indexation_tables.ipynb"><< 1. Indexer des données</a></td>
        <td style="text-align:right;"><a href="3_tri.ipynb">3. Trier des données >></a></td>
    </tr>
</table>

In [ ]:
import os
import csv
import requests
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import MiniMap, MarkerCluster

URL = "https://www.data.gouv.fr/fr/datasets/r/d7f3ddf4-2225-4ac2-9a1d-26971ce92969"


def display_map(url):
    """Affiche la carte des données qui se trouvent à 'url'
    """
    local_file = "geolocalisation.csv"
    if not local_file in os.listdir():
        data = requests.get(url).content
        with open(local_file, 'wb') as csvfile:
            csvfile.write(data)
    with open(local_file) as csvfile:
        m = Map(location=[0,0], width="%100", height="%100", zoom_start=10)
        #marker_cluster = MarkerCluster().add_to(m)
        minimap = MiniMap()
        m.add_child(minimap)
        # Creation des clusters et des couches
        gratuit_layer = folium.FeatureGroup(name="Gratuit")
        gratuit_cluster = MarkerCluster().add_to(gratuit_layer)
        m.add_child(gratuit_layer)

        payant_layer = folium.FeatureGroup(name="Payant")
        payant_cluster = MarkerCluster().add_to(payant_layer)
        m.add_child(payant_layer)

        unknown_layer = folium.FeatureGroup(name="Tarifs non communiqués")
        unknown_cluster = MarkerCluster().add_to(unknown_layer)        
        m.add_child(unknown_layer)

        libre_participation_layer = folium.FeatureGroup(name="Libre participation")
        libre_participation_cluster = MarkerCluster().add_to(libre_participation_layer)        
        m.add_child(libre_participation_layer)
        m.add_child(folium.map.LayerControl())

        
        reader = csv.DictReader(csvfile, delimiter=';')
        somme_latitude = 0
        somme_longitude = 0
        compteur = 0
        for row in reader:
            code_postal = row['detailidentadressecp']
            code_postal = int(''.join(code_postal.split(' ')))
            if code_postal in range(29200,29201):
                compteur += 1
                latitude = row['gmaplatitude']
                longitude = row['gmaplongitude']
                somme_latitude += float(latitude)
                somme_longitude += float(longitude)
                tooltip = row['syndicobjectname']
                icon = get_icon(row)
                if row['tarifentree'] == 'Gratuit':
                    Marker([latitude, longitude], icon=icon, popup="<i>Hello</i>", tooltip=tooltip).add_to(gratuit_cluster)
                elif row['tarifentree'] == 'Payant':
                    Marker([latitude, longitude], icon=icon, popup="<i>Hello</i>", tooltip=tooltip).add_to(payant_cluster)
                elif row['tarifentree'] == 'Tarifs non communiqués':
                    Marker([latitude, longitude], icon=icon, popup="<i>Hello</i>", tooltip=tooltip).add_to(unknown_cluster)
                else:
                    Marker([latitude, longitude], icon=icon, popup="<i>Hello</i>", tooltip=tooltip).add_to(libre_participation_layer)

                
        moy_latitude = somme_latitude/compteur
        moy_longitude = somme_longitude/compteur
        m.location = (moy_latitude, moy_longitude)
        return m
    
def get_icon(row):
    """
    Retourne l'icone adequat
    """
    if row['tarifentree'] == 'Gratuit':
        icon=folium.Icon(color='red', icon='home', prefix='fa')
    elif row['tarifentree'] == 'Payant':
        icon=folium.Icon(color='green', icon='home', prefix='fa')
    elif row['tarifentree'] == 'Tarifs non communiqués':
        icon=folium.Icon(color='blue', icon='home', prefix='fa')
    else:
        icon=folium.Icon(color='black', icon='home', prefix='fa')
    return icon
        


m = display_map(URL)
m

[Retour au sommaire](../index.ipynb)